### Every thing is same here but just logically we are storing previous history as a list in a state

In [6]:
from gen_ai_hub.proxy.langchain.init_models import init_llm

model = init_llm(model_name = "gpt-4o")

model.invoke("Are you there?").content

"Yes, I'm here! How can I assist you today?"

### Here message is a list either contains prev human message or AI message

In [1]:
from typing import TypedDict
from langchain_core.messages import HumanMessage, AIMessage
from typing import Union, List
class AgentState(TypedDict):
    message: List[Union[HumanMessage, AIMessage]] # List[HumanMessage | AIMessage]


### Here are along with generation we are appending current state response as well

In [2]:
def generate_response(state:AgentState) -> AgentState:
    response = model.invoke(state["message"])
    state["message"].append(AIMessage(content=response.content))
    print(response.content)
    return state


In [3]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(state_schema=AgentState)
graph_builder.add_node("generate", generate_response)


graph_builder.add_edge(START, "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [8]:
conversational_history = []
while True:
    query = input("QUERY: ")
    if query.lower() == "exit":break
    conversational_history.append(HumanMessage(content=query))
    response = graph.invoke({"message": conversational_history})
    conversational_history = response["message"]
    print(response)
    

Hello! How can I assist you today?
{'message': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={})]}
Hi Rakesh! Nice to meet you. How can I help you today?
{'message': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='im rakesh', additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Rakesh! Nice to meet you. How can I help you today?', additional_kwargs={}, response_metadata={})]}
You mentioned that your name is Rakesh. How can I assist you further?
{'message': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='im rakesh', additional_kwargs={}, response_metadata={}), A

### Store it somewhere if you want to use this later


In [9]:
with open("logging.txt", "w") as file:
    file.write("Your Conversation Log:\n")
    
    for message in conversational_history:
        if isinstance(message, HumanMessage):
            file.write(f"You: {message.content}\n")
        elif isinstance(message, AIMessage):
            file.write(f"AI: {message.content}\n\n")
    file.write("End of Conversation")